In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

#needed for jupyter notebook
#https://github.com/tensorflow/tensorflow/issues/24828
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Num GPUs Available:  3


In [2]:
# import numpy as np
# import tensorflow as tf

# https://github.com/jkjung-avt/keras-cats-dogs-tutorial/blob/master/train_resnet50.py
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [36]:
"""
hyperparameters + filenames
"""
DATASET_PATH = "data"

# doubled min width in dataset
# height = 2*width
# height, width 
# try reducing batch size or freeze more layers if your GPU runs out of memory

TARGET_SIZE = (224,112) 
BATCH_SIZE = 64  
NUM_EPOCHS = 200
# NUM_CLASSES defined later

In [37]:
"""
augmenting data
"""
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    shear_range=30,

    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,

    brightness_range=(0.75, 1.0),
    channel_shift_range=75.0,

    horizontal_flip=True)

train_batches = train_datagen.flow_from_directory(
    DATASET_PATH + "/train",
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
#     interpolation='bicubic',
    class_mode='categorical',
    shuffle=True)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

valid_batches = valid_datagen.flow_from_directory(
    DATASET_PATH + "/valid",
    target_size=TARGET_SIZE, 
    batch_size=BATCH_SIZE,
#     interpolation='bicubic',
    class_mode='categorical',
    shuffle=True)

NUM_CLASSES = len(train_batches.class_indices.items())

"""
training nnet
"""
# https://github.com/jkjung-avt/keras-cats-dogs-tutorial/blob/master/train_resnet50.py

#train net
# build our classifier model based on pre-trained ResNet50:
# 1. we don't include the top (fully connected) layers of ResNet50
# 2. we add a DropOut layer followed by a Dense (fully connected)
#    layer which generates softmax class score for each class
# 3. we compile the final model using an Adam optimizer, with a
#    low learning rate (since we are 'fine-tuning')
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(TARGET_SIZE[0],TARGET_SIZE[1],3))
x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)

"""
#freeze layers?
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True
"""

net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])
print(net_final.summary())

# train the model
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

Found 622 images belonging to 13 classes.
Found 152 images belonging to 13 classes.
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 224, 112, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 118, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 56, 64)  9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 56, 64)  256         conv1_conv[0][0]                 
________

Epoch 1/200
9/9 [==============================] - 13s 1s/step - loss: 4.7766 - acc: 0.1147 - val_loss: 3.5454 - val_acc: 0.1719
Epoch 2/200
9/9 [==============================] - 3s 359ms/step - loss: 4.0189 - acc: 0.1997 - val_loss: 3.3939 - val_acc: 0.2891
Epoch 3/200
9/9 [==============================] - 3s 332ms/step - loss: 4.0501 - acc: 0.2401 - val_loss: 3.2354 - val_acc: 0.3125
Epoch 4/200
9/9 [==============================] - 3s 331ms/step - loss: 3.8366 - acc: 0.2833 - val_loss: 3.0263 - val_acc: 0.3125
Epoch 5/200
9/9 [==============================] - 3s 316ms/step - loss: 3.7270 - acc: 0.2832 - val_loss: 2.8086 - val_acc: 0.3281
Epoch 6/200
9/9 [==============================] - 3s 322ms/step - loss: 3.3776 - acc: 0.3194 - val_loss: 2.6629 - val_acc: 0.3594
Epoch 7/200
9/9 [==============================] - 3s 321ms/step - loss: 3.3285 - acc: 0.3208 - val_loss: 2.5745 - val_acc: 0.3672
Epoch 8/200
9/9 [==============================] - 3s 333ms/step - loss: 3.1080 - acc

Epoch 76/200
9/9 [==============================] - 3s 329ms/step - loss: 1.1385 - acc: 0.6774 - val_loss: 0.6979 - val_acc: 0.7734
Epoch 77/200
9/9 [==============================] - 3s 341ms/step - loss: 1.1503 - acc: 0.7101 - val_loss: 0.7058 - val_acc: 0.7812
Epoch 78/200
9/9 [==============================] - 3s 333ms/step - loss: 1.0068 - acc: 0.7043 - val_loss: 0.7011 - val_acc: 0.7891
Epoch 79/200
9/9 [==============================] - 3s 330ms/step - loss: 0.9996 - acc: 0.7079 - val_loss: 0.6717 - val_acc: 0.7812
Epoch 80/200
9/9 [==============================] - 3s 311ms/step - loss: 0.9808 - acc: 0.7186 - val_loss: 0.6517 - val_acc: 0.7891
Epoch 81/200
9/9 [==============================] - 3s 335ms/step - loss: 0.9148 - acc: 0.7276 - val_loss: 0.6327 - val_acc: 0.7891
Epoch 82/200
9/9 [==============================] - 3s 341ms/step - loss: 1.0271 - acc: 0.7205 - val_loss: 0.6238 - val_acc: 0.7891
Epoch 83/200
9/9 [==============================] - 3s 339ms/step - loss: 0.

9/9 [==============================] - 3s 325ms/step - loss: 0.5156 - acc: 0.8405 - val_loss: 0.4845 - val_acc: 0.8438
Epoch 151/200
9/9 [==============================] - 3s 345ms/step - loss: 0.4461 - acc: 0.8513 - val_loss: 0.4848 - val_acc: 0.8438
Epoch 152/200
9/9 [==============================] - 3s 347ms/step - loss: 0.4556 - acc: 0.8674 - val_loss: 0.4821 - val_acc: 0.8438
Epoch 153/200
9/9 [==============================] - 3s 343ms/step - loss: 0.5219 - acc: 0.8369 - val_loss: 0.4775 - val_acc: 0.8516
Epoch 154/200
9/9 [==============================] - 3s 317ms/step - loss: 0.4152 - acc: 0.8710 - val_loss: 0.4733 - val_acc: 0.8438
Epoch 155/200
9/9 [==============================] - 3s 344ms/step - loss: 0.6035 - acc: 0.8177 - val_loss: 0.4732 - val_acc: 0.8594
Epoch 156/200
9/9 [==============================] - 3s 309ms/step - loss: 0.5309 - acc: 0.8407 - val_loss: 0.4778 - val_acc: 0.8516
Epoch 157/200
9/9 [==============================] - 3s 331ms/step - loss: 0.5372 -

In [38]:
"""
fill this in
"""
acc = 8799

import time, os
MODELS_PATH = "saved_models"
if not os.path.exists(MODELS_PATH):
    os.mkdir(MODELS_PATH)
    
model_filename = "{}_{}_highangle_allclass_RES50.h5".format(acc, round(time.time()))
filepath = os.path.join(MODELS_PATH, model_filename)
net_final.save(filepath)
print("Model saved at:", filepath)

Model saved at: saved_models/8799_1575782539_highangle_allclass_RES50.h5
